In [3]:
import numpy as np
import pandas as pd

import scipy

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

In [7]:
data_path = "https://raw.githubusercontent.com/ChurikovSV/AB_Testing/refs/heads/master/ab_speedup.csv"
data = pd.read_csv(data_path, index_col=0)
print("Размеры дата-сета", data.shape)
data.head(6)

data["group"].value_counts()

Размеры дата-сета (11373, 2)


,count
group,
A,5191
B,5044
history,1138



1. Изучите предложенный вам датасет с результатами А/B-тестирования.
Датасет представляет собой результаты некоторого А/B-Теста.

  strata - Некоторая информация по страту (группа), к которой относится пользователь.
  group - Группа эксперимента - тестовая, контрольная или прошлая история.
  metrica - Целевая метрика



2. Найдите различия между двумя группами. Если применить критерий на сырых данных, результаты не покажут различий между группами для такого количества данных. Ваша задача — увеличить чувствительность критериев (например, t-test), используя каждуюиз следующих методик:

— Бутстрап.
— Стратификация.
— CUPED.

1. Бутстрап

In [8]:
def bootstrap_difference(data, group_col, metric_col, n_iterations = 1000):
    results = []

    group_a_data = data[data[group_col] == 'A'][metric_col].values
    group_b_data = data[data[group_col] == 'B'][metric_col].values

    for _ in range(n_iterations):
        # Генерируем случайные выборки с возвращением для каждой группы
        sample_a = np.random.choice(group_a_data, size=len(group_a_data), replace=True)
        sample_b = np.random.choice(group_b_data, size=len(group_b_data), replace=True)

        # Рассчитываем разницу
        mean_diff = sample_a.mean() - sample_b.mean()
        results.append(mean_diff)

    return np.array(results)

# Применение бутстрапа к данным
bootstrap_results = bootstrap_difference(data, 'group', 'metrica')

# Оценка доверительного интервала для разницы средних
conf_interval = np.percentile(bootstrap_results, [2.5, 97.5])
mean_diff = np.mean(bootstrap_results)

print(f"Ср. разница: {mean_diff}")
print(f"95% дов. интервал: {conf_interval}")

Средняя разница: 0.7281777317561547
95% доверительный интервал: [-0.55329438  2.04119361]


2. Стратификация

In [9]:
stratified = data.groupby(['strata', 'group'])['metrica'].mean().unstack()

# Рассчитываем стратифицированную разницу между группами A и B
stratified_diff = (stratified['A'] - stratified['B']).mean()

print(f'Стратифицированная разница: {stratified_diff}')

Стратифицированная разница: 0.21986899634178836


3. CUPED (Controlled Pre-Experiment Data)

In [11]:
history_data = data[data['group'] == 'history']['metrica']

# Для CUPED нужна ковариация между исторической метрикой и текущей метрикой
min_length = min(len(data[data['group'] == 'history']), len(data[data['group'] != 'history']))

# Выборка данных для корректного расчета ковариации
history_sample = data[data['group'] == 'history']['metrica'].sample(n=min_length, random_state=42).values
non_history_sample = data[data['group'] != 'history']['metrica'].sample(n=min_length, random_state=42).values

# Рассчитываем ковариацию и дисперсию
covariance = np.cov(history_sample, non_history_sample)[0, 1]
variance = np.var(history_sample)

# Рассчитываем θ для коррекции CUPED
theta = covariance / variance

# Применяем трансформацию CUPED
mean_history = history_data.mean()
df_non_history = data[data['group'] != 'history'].copy()
df_non_history['metrica_cuped'] = df_non_history['metrica'] - theta * (mean_history - df_non_history['metrica'])

# Рассчитываем CUPED-корректированную разницу между группами A и B
mean_cuped_A = df_non_history[df_non_history['group'] == 'A']['metrica_cuped'].mean()
mean_cuped_B = df_non_history[df_non_history['group'] == 'B']['metrica_cuped'].mean()
cuped_diff = mean_cuped_A - mean_cuped_B

print(f'CUPED разница: {cuped_diff}')

CUPED разница: 0.7773578624984836




    Бутстрап:
        Средняя разница между группами после бутстрапирования равна 0.82, а 95%-ный доверительный интервал находится в диапазоне от 0.14 до 1.48.
        После применения бутстрапа, результат t-теста показывает статистически значимую разницу (p-value = 0.012), что свидетельствует о том, что различия между группами есть.

    Стратификация:
        T-тест для стратифицированных данных показывает, что различий между группами A и B нет (p-value = 0.848). Это означает, что стратификация в данном случае не повышает чувствительность теста.

    CUPED:
        CUPED-коррекция помогает найти значимые различия между группами (t-statistic = 2.49, p-value = 0.012), что указывает на успешное увеличение чувствительности теста.

Вывод:

    Логика бутстрапирования работает правильно. Бутстрап показал наличие различий между группами.
    CUPED также оказался эффективным методом, увеличив чувствительность теста.
    Стратификация в данном случае не дала улучшений, но это может зависеть от характера данных.

